$\newcommand{norm}[1]{||#1||}$
$\newcommand{mbx}{\mathbf x}$
$\newcommand{mbu}{\mathbf u}$
$\newcommand{d}{\text{div}}$
$\newcommand{mbf}{\mathbf f}$
$\newcommand{t}{\text{tr }}$
$\newcommand{mcA}{\mathcal A}$

# 线弹性问题

## 模型

$$
-\mu\Delta\mbu - (\lambda+\mu)\nabla\d\mbu = \mbf
$$

$$
A(\mbu) = \frac{\nabla\mbu + \nabla\mbu^T}{2}
$$

$$
\sigma(\bu) = 2\mu A(\mbu) + \lambda(\t A(\mbu)) I
$$

$$
\mcA(\sigma) = \frac{1}{2\mu}\left(\sigma - \frac{\lambda}{n\lambda + 2\mu}(\t\sigma) I\right)
$$

$$
-\d\sigma(\bu) = \f
$$

## 存在问题

* 解法器效率太低
* 矩阵组装基本是线性复杂度，但仍有改进空间
* 需要更多测试例子

## 测试模型数据

In [6]:
from IPython.display import display
from sympy import *
import numpy as np
init_printing()

def linear_elasticity_model(u, v):
    n = len(v)
    d, e = symbols('lam, mu')
    du = u.jacobian(v)
    sigma = d*du.trace()*eye(n)+e*(du + du.transpose())
    f = -diff(sigma[:, 0], v[0])
    for i in range(1, n):
        f -= diff(sigma[:, i], v[i])
    return f, du, sigma

**例 1** 求解区域为 $\Omega = [0, 1]^3$
$$
\mbu = \begin{pmatrix}
2^4 \\ 2^5 \\ 2^6
\end{pmatrix}
x(1-x)y(1-y)z(1-z)
$$

In [3]:
x, y, z, a, b, c= symbols('x, y, z, a, b, c')
u0 = x*(1-x)*y*(1-y)*z*(1-z)
u = Matrix([a, b, c])*u0
f = linear_elasticity_model(u, (x, y, z))
print(f)

(Matrix([
[4*a*mu*y*z*(-y + 1)*(-z + 1) - lam*(-2*a*y*z*(-y + 1)*(-z + 1) + b*x*y*z*(-z + 1) - b*x*z*(-y + 1)*(-z + 1) - b*y*z*(-x + 1)*(-z + 1) + b*z*(-x + 1)*(-y + 1)*(-z + 1) + c*x*y*z*(-y + 1) - c*x*y*(-y + 1)*(-z + 1) - c*y*z*(-x + 1)*(-y + 1) + c*y*(-x + 1)*(-y + 1)*(-z + 1)) - mu*(-2*a*x*y*(-x + 1)*(-y + 1) + c*x*y*z*(-y + 1) - c*x*y*(-y + 1)*(-z + 1) - c*y*z*(-x + 1)*(-y + 1) + c*y*(-x + 1)*(-y + 1)*(-z + 1)) - mu*(-2*a*x*z*(-x + 1)*(-z + 1) + b*x*y*z*(-z + 1) - b*x*z*(-y + 1)*(-z + 1) - b*y*z*(-x + 1)*(-z + 1) + b*z*(-x + 1)*(-y + 1)*(-z + 1))],
[  4*b*mu*x*z*(-x + 1)*(-z + 1) - lam*(a*x*y*z*(-z + 1) - a*x*z*(-y + 1)*(-z + 1) - a*y*z*(-x + 1)*(-z + 1) + a*z*(-x + 1)*(-y + 1)*(-z + 1) - 2*b*x*z*(-x + 1)*(-z + 1) + c*x*y*z*(-x + 1) - c*x*y*(-x + 1)*(-z + 1) - c*x*z*(-x + 1)*(-y + 1) + c*x*(-x + 1)*(-y + 1)*(-z + 1)) - mu*(a*x*y*z*(-z + 1) - a*x*z*(-y + 1)*(-z + 1) - a*y*z*(-x + 1)*(-z + 1) + a*z*(-x + 1)*(-y + 1)*(-z + 1) - 2*b*y*z*(-y + 1)*(-z + 1)) - mu*(-2*b*x*y*(-x + 1)*(-y 

Construct linear system time: 1.659951514000113
Solve time: 0.08165926999981821
Construct linear system time: 13.403858605000096
Solve time: 2.987838785999884
Construct linear system time: 108.33645204000004
Solve time: 192.936152407
Ndof: [  750  4374 29478]
error: [[  1.39739596e+00   2.29570086e-01   2.73310407e-02]
 [  1.46080383e+01   4.92832365e+00   1.25911061e+00]
 [  9.33096445e-02   8.14716287e-03   5.08085642e-04]]

**例 2** 求解区域为 $\Omega = [0, 1]^3$

$$
u = \begin{pmatrix}
e^{x-y}x(1-x)y(1-y) \\
\sin(\pi x)\sin(\pi y)
\end{pmatrix}
$$

In [7]:
x, y = symbols('x, y')
u = Matrix([exp(x-y)*x*(1-x)*y*(1-y), sin(pi*x)*sin(pi*y)])
f, du, sigma= linear_elasticity_model(u, (x,y))
ss, cc, e, t0 = symbols('ss, cc, e, t0')
sub = {sin(pi*x)*sin(pi*y):ss, cos(pi*x)*cos(pi*y):cc, exp(x-y):e, (-x+1)*(-y+1):t0}
print(f)
print(f.subs(sub))
print(du)

Matrix([[-lam*(x*y*(-x + 1)*(-y + 1)*exp(x - y) - 2*x*y*(-y + 1)*exp(x - y) + 2*y*(-x + 1)*(-y + 1)*exp(x - y) - 2*y*(-y + 1)*exp(x - y) + pi**2*cos(pi*x)*cos(pi*y)) - mu*(2*x*y*(-x + 1)*(-y + 1)*exp(x - y) - 4*x*y*(-y + 1)*exp(x - y) + 4*y*(-x + 1)*(-y + 1)*exp(x - y) - 4*y*(-y + 1)*exp(x - y)) - mu*(x*y*(-x + 1)*(-y + 1)*exp(x - y) + 2*x*y*(-x + 1)*exp(x - y) - 2*x*(-x + 1)*(-y + 1)*exp(x - y) - 2*x*(-x + 1)*exp(x - y) + pi**2*cos(pi*x)*cos(pi*y))], [-lam*(-x*y*(-x + 1)*(-y + 1)*exp(x - y) - x*y*(-x + 1)*exp(x - y) + x*y*(-y + 1)*exp(x - y) + x*y*exp(x - y) + x*(-x + 1)*(-y + 1)*exp(x - y) - x*(-y + 1)*exp(x - y) - y*(-x + 1)*(-y + 1)*exp(x - y) - y*(-x + 1)*exp(x - y) + (-x + 1)*(-y + 1)*exp(x - y) - pi**2*sin(pi*x)*sin(pi*y)) - mu*(-x*y*(-x + 1)*(-y + 1)*exp(x - y) - x*y*(-x + 1)*exp(x - y) + x*y*(-y + 1)*exp(x - y) + x*y*exp(x - y) + x*(-x + 1)*(-y + 1)*exp(x - y) - x*(-y + 1)*exp(x - y) - y*(-x + 1)*(-y + 1)*exp(x - y) - y*(-x + 1)*exp(x - y) + (-x + 1)*(-y + 1)*exp(x - y) - pi**